In [1]:
# Dependencies
import pandas as pd
import numpy as np
import csv
from datetime import datetime as dt
import unicodecsv

### Loading CSV files

In [2]:
# Define a function to read csv files
def read_csv(file_name):
    with open(file_name, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)

In [3]:
enrollments = read_csv("enrollments.csv")
daily_engagement = read_csv("daily_engagement.csv")
project_submissions = read_csv("project_submissions.csv")

In [4]:
enrollments[0]

OrderedDict([('account_key', '448'),
             ('status', 'canceled'),
             ('join_date', '2014-11-10'),
             ('cancel_date', '2015-01-14'),
             ('days_to_cancel', '65'),
             ('is_udacity', 'True'),
             ('is_canceled', 'True')])

In [5]:
daily_engagement[0]

OrderedDict([('acct', '0'),
             ('utc_date', '2015-01-09'),
             ('num_courses_visited', '1.0'),
             ('total_minutes_visited', '11.6793745'),
             ('lessons_completed', '0.0'),
             ('projects_completed', '0.0')])

In [6]:
project_submissions[0]

OrderedDict([('creation_date', '2015-01-14'),
             ('completion_date', '2015-01-16'),
             ('assigned_rating', 'UNGRADED'),
             ('account_key', '256'),
             ('lesson_key', '3176718735'),
             ('processing_state', 'EVALUATED')])

### Fixing Data Types

In [7]:
# function for fixing the datetime type
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
# function for fixing the object data type
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

In [8]:
# Clean up the enrollement data
for enrollment in enrollments:
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_udacity'] = enrollment['is_udacity']=="True"
    enrollment['is_canceled'] = enrollment['is_canceled']=="True" 

In [9]:
enrollments[0]

OrderedDict([('account_key', '448'),
             ('status', 'canceled'),
             ('join_date', datetime.datetime(2014, 11, 10, 0, 0)),
             ('cancel_date', datetime.datetime(2015, 1, 14, 0, 0)),
             ('days_to_cancel', 65),
             ('is_udacity', True),
             ('is_canceled', True)])

In [10]:
# Clean up the daily_engagement data
for engagement in daily_engagement:
    engagement['utc_date'] = parse_date(engagement['utc_date'])
    engagement['lessons_completed'] = int(float(engagement['lessons_completed']))
    engagement['num_courses_visited'] = int(float(engagement['num_courses_visited']))
    engagement['projects_completed'] = int(float(engagement['projects_completed']))
    engagement['total_minutes_visited'] = float(engagement['total_minutes_visited'])    

In [11]:
# Clean up the project_submissions data
for submissions in project_submissions:
    submissions['creation_date'] = parse_date(submissions['creation_date'])
    submissions['completion_date'] = parse_date(submissions['completion_date'])   

In [12]:
# Let's check how many rows we have in the tables
def num_rows(data):
    return len(data)

In [13]:
enrollment_num_row = num_rows(enrollments)
engagement_num_row = num_rows(daily_engagement)
submission_num_row = num_rows(project_submissions)

In [14]:
print('Total number of rows for enrollments table is {}'.format(enrollment_num_row))
print('Total number of rows for daily engagement table is {}'.format(engagement_num_row))
print('Total number of rows for project submissions table is {}'.format(submission_num_row))

Total number of rows for enrollments table is 1640
Total number of rows for daily engagement table is 136240
Total number of rows for project submissions table is 3642


In [15]:
# Let's change the column name of 'acct' to 'account_key' in engagement table
for column in daily_engagement:
    column['account_key'] = column['acct']
    del[column['acct']]

In [16]:
daily_engagement

[OrderedDict([('utc_date', datetime.datetime(2015, 1, 9, 0, 0)),
              ('num_courses_visited', 1),
              ('total_minutes_visited', 11.6793745),
              ('lessons_completed', 0),
              ('projects_completed', 0),
              ('account_key', '0')]),
 OrderedDict([('utc_date', datetime.datetime(2015, 1, 10, 0, 0)),
              ('num_courses_visited', 2),
              ('total_minutes_visited', 37.2848873333),
              ('lessons_completed', 0),
              ('projects_completed', 0),
              ('account_key', '0')]),
 OrderedDict([('utc_date', datetime.datetime(2015, 1, 11, 0, 0)),
              ('num_courses_visited', 2),
              ('total_minutes_visited', 53.6337463333),
              ('lessons_completed', 0),
              ('projects_completed', 0),
              ('account_key', '0')]),
 OrderedDict([('utc_date', datetime.datetime(2015, 1, 12, 0, 0)),
              ('num_courses_visited', 1),
              ('total_minutes_visited', 33.4892

In [17]:
# Let's check the number of unique students of each table (account keys)
def find_unique_students(data):
    unique_students = set()
    for data_point in data:
        unique_students.add(data_point['account_key'])
    return unique_students

In [18]:
enrollment_unique_student = find_unique_students(enrollments)
engagement_unique_student = find_unique_students(daily_engagement)
submission_unique_student = find_unique_students(project_submissions)

In [19]:
print('The number of unique students in enrollment table is {}'.format(len(enrollment_unique_student)))
print('The number of unique students in daily engagement table is {}'.format(len(engagement_unique_student)))
print('The number of unique students in project submissions table is {}'.format(len(submission_unique_student)))

The number of unique students in enrollment table is 1302
The number of unique students in daily engagement table is 1237
The number of unique students in project submissions table is 743


We can see there're different number of unique students in tables and there should be a problem there.


In [43]:
# Let's find out the students who are not in engagement unique table.

enrollment_who_not_in_engagement_unique = []

for enrollment in enrollments:
    student = enrollment['account_key']
    if student in engagement_unique_student:
        continue
    else:
        enrollment_who_not_in_engagement_unique.append(enrollment) 
        
enrollment_who_not_in_engagement_unique

[OrderedDict([('account_key', '1219'),
              ('status', 'canceled'),
              ('join_date', datetime.datetime(2014, 11, 12, 0, 0)),
              ('cancel_date', datetime.datetime(2014, 11, 12, 0, 0)),
              ('days_to_cancel', 0),
              ('is_udacity', False),
              ('is_canceled', True)]),
 OrderedDict([('account_key', '871'),
              ('status', 'canceled'),
              ('join_date', datetime.datetime(2014, 11, 13, 0, 0)),
              ('cancel_date', datetime.datetime(2014, 11, 13, 0, 0)),
              ('days_to_cancel', 0),
              ('is_udacity', False),
              ('is_canceled', True)]),
 OrderedDict([('account_key', '1218'),
              ('status', 'canceled'),
              ('join_date', datetime.datetime(2014, 11, 15, 0, 0)),
              ('cancel_date', datetime.datetime(2014, 11, 15, 0, 0)),
              ('days_to_cancel', 0),
              ('is_udacity', False),
              ('is_canceled', True)]),
 OrderedDict([('a

As we can there are lot of poeple have same date of join and cancel date!!

In [49]:
# let's check the students who they have different join date and cancel date

num_student = 0

for enrollment in enrollments:
    student = enrollment['account_key']
    if student not in engagement_unique_student and \
    enrollment['join_date'] != enrollment['cancel_date']:
        num_student+=1
        print(enrollment)

print(num_student)

OrderedDict([('account_key', '1304'), ('status', 'canceled'), ('join_date', datetime.datetime(2015, 1, 10, 0, 0)), ('cancel_date', datetime.datetime(2015, 3, 10, 0, 0)), ('days_to_cancel', 59), ('is_udacity', True), ('is_canceled', True)])
OrderedDict([('account_key', '1304'), ('status', 'canceled'), ('join_date', datetime.datetime(2015, 3, 10, 0, 0)), ('cancel_date', datetime.datetime(2015, 6, 17, 0, 0)), ('days_to_cancel', 99), ('is_udacity', True), ('is_canceled', True)])
OrderedDict([('account_key', '1101'), ('status', 'current'), ('join_date', datetime.datetime(2015, 2, 25, 0, 0)), ('cancel_date', None), ('days_to_cancel', None), ('is_udacity', True), ('is_canceled', False)])
3


In [52]:
# Check the account keys for all Udacity test accounts
udacity_accounts = set()
for udacity in enrollments:
    if udacity["is_udacity"] == True:
        udacity_accounts.add(udacity["account_key"])
print(udacity_accounts)

{'818', '448', '1069', '312', '1101', '1304'}


In [ ]:
# Given some data with an account_key field, removes any records corresponding to Udacity test accounts
def remove_udacity_acoounts(data):
    non_udacity_accounts = []
    